In [7]:
import geopandas as gpd
import pandas as pd
import geoplot as gplt
from shapely.geometry import Polygon
from shapely.ops import unary_union

from zipfile import ZipFile

# Carribean Sport project
One time script to clean up the raw data for the caribbean cricket and baseball data sets

In [2]:
wm = gpd.read_file('../maps/ne_10m_admin_0_countries_lakes.shp').set_index('NAME')

countries_to_ignore = [
    'United States of America',
    'Mexico',
    'Nicaragua',
    'Costa Rica',
    'Panama',
    'Colombia',
    'Venezuela',
    'Suriname',
    'Brazil',
]

ignore_countries = wm[wm.index.isin(countries_to_ignore)].geometry.unary_union

florida_mask = Polygon(
    (
        (-79.483220, 24.367631),     
        (-84.360623, 24.367631,),    
        (-84.360623, 29.596543,),     
        (-79.483220, 29.596543)
    )
)
colombia_mask_1 = Polygon(
    [
        [-61.812529, 10.848645],
        [-61.812529, 10.173257],
        [-62.700268, 10.173257,],
        [-62.700268, 10.848645]
    ]
)
colombia_mask_2 = Polygon(
    (
        (-62.565606, 11.333120),
        (-68.496503, 11.333120),
        (-68.496503, 8.484943),        
        (-62.565606, 8.484943)
    )
)

ignore_mask = unary_union([ignore_countries, florida_mask,
                          colombia_mask_1, colombia_mask_2])

In [3]:
cd = gpd.read_file('../data/caribbean_cricket_raw.geojson')
print('cricket\n',len(cd))
cd = cd[~cd['geometry'].within(ignore_mask)]
print('new', len(cd))
cd.to_file('../data/caribbean_cricket.geojson', driver='GeoJSON')

cricket
 219
new 213


In [4]:
bd1 = gpd.read_file('../data/caribbean_baseball_1_raw.geojson')
bd2 = gpd.read_file('../data/caribbean_baseball_2_raw.geojson')
bd = gpd.GeoDataFrame(pd.concat([bd1,bd2]))
print('Baseball\n',len(bd))
bd = bd.drop_duplicates()
bd = bd[~bd['geometry'].within(ignore_mask)]
print(len(bd))

bd.to_file('../data/caribbean_baseball.geojson', driver='GeoJSON')

Baseball
 4845
2974


# McDonald's Project

In [8]:
with ZipFile('../data/McDonalds.geojson.zip') as myzip:
    with myzip.open('McDonalds.geojson') as myfile:
        gdf = gpd.read_file(myfile)
    

In [10]:
world_map = gpd.read_file('../maps/ne_10m_admin_1_states_provinces_lakes.shp')
us_states = world_map.query('admin == "United States of America"').set_index('name').drop(['Alaska','Hawaii'])
mask = us_states.geometry.unary_union

In [11]:
gdf = gdf[gdf['geometry'].within(mask)]

In [21]:
with ZipFile('../data/McDonalds.geojson.zip', 'w') as myzip:
    with myzip.open('McDonalds.geojson') as myfile:
        gdf.to_file(myfile, driver='GeoJSON')

KeyError: "There is no item named 'McDonalds.geojson' in the archive"

In [15]:
gdf.to_pickle('test.pkl')

In [26]:
gdf

,id,@id,amenity,brand,cuisine,landuse,name,ref,type,brand:wikidata,...,payment:union_pay,name:nl,fax,addr:alt_street,changing_table:count,bin,drive_through:lanes,delivery:covid19,website:branch,geometry
0,relation/1991997,relation/1991997,fast_food,McDonald's,burger,retail,McDonald's,12298,multipolygon,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-81.42254 28.50874, -81.42259 28.508..."
1,relation/2820769,relation/2820769,fast_food,McDonald's,burger,retail,McDonald's,10138,multipolygon,Q38076,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-81.36464 28.45257, -81.36464 28.452..."
2,relation/5842438,relation/5842438,fast_food,None,burger,None,McDonald's,None,multipolygon,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-81.65690 35.72363, -81.65689 35.723..."
3,relation/6136951,relation/6136951,fast_food,McDonald's,burger,None,McDonald's,None,multipolygon,Q38076,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-111.10052 44.65748, -111.1005..."
4,relation/6691429,relation/6691429,fast_food,McDonald's,burger,None,McDonald's,None,multipolygon,Q38076,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-117.26358 47.65657, -117.26358 47.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10254,node/7820523100,node/7820523100,fast_food,McDonald's,burger,None,McDonald's,None,None,Q38076,...,None,None,None,None,None,None,None,None,None,POINT (-91.05774 30.43248)
10255,node/7820523254,node/7820523254,fast_food,McDonald's,burger,None,McDonald's,None,None,Q38076,...,None,None,None,None,None,None,None,None,None,POINT (-91.09051 30.41797)
10256,node/7820545870,node/7820545870,fast_food,McDonald's,burger,None,McDonald's,None,None,Q38076,...,None,None,None,None,None,None,None,None,None,POINT (-80.35105 41.89071)
10257,node/7820859478,node/7820859478,fast_food,McDonald's,burger,None,McDonald's,None,None,Q38076,...,None,None,None,None,None,None,None,None,None,POINT (-80.83303 35.14868)
